# _Physician Compare National: Explore #5_

This notebook is a continuation from of my analysis on the following data gathered via [Data.Medicare.gov](https://data.medicare.gov/Physician-Compare/Physician-Compare-National-Downloadable-File/mj5m-pzi6). It contains general information about individual eligible professionals (EPs) such as demographic information and Medicare quality program participation. This dataset is updated twice a month with the most current demographic information available at that time.

# _Today's Goal_

My primary goal is to intial cleaning of the data set. There are still multiple columns I have not looked at so further exploration into the unique values within each column, and any missing values, need to be assessed and dealth with.

A secondary goal is to figure out a way to better format the zip codes column; further, I will look for a way to add information regarding county level data, whether that is through ZIP or FIPS codes. Time permitting, I could also begin initial trials of putting together a visual via `plot.ly`.

In [1]:
# first thing we need to do --> load in the data
# import pandas
import pandas as pd
pd.options.display.max_columns = None
%load_ext autoreload
%autoreload 2

# import data from yesterday
data = pd.read_csv('physician_compare_national-updates-1.csv', low_memory=False);

# inspect the first five rows
data.head()

,npi,ind_pac_id,ind_enrl_id,full_nm,gndr,cred,med_sch,grd_yr,pri_spec,sec_spec_1,sec_spec_2,sec_spec_3,sec_spec_4,sec_spec_all,org_lgl_nm,org_pac_id,num_org_mem,adr_ln_1,adr_ln_2,ln_2_sprs,cty,st,zip,phn_numbr,hosp_afl_1,hosp_afl_lbn_1,hosp_afl_2,hosp_afl_lbn_2,hosp_afl_3,hosp_afl_lbn_3,hosp_afl_4,hosp_afl_lbn_4,hosp_afl_5,hosp_afl_lbn_5,assgn
0,1003000126,7517003643,I20130530000085,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,EMERGENCY MEDICINE ASSOCIATES PA PC,8.022915e+09,182.0,1850 TOWN CTR PKWY,NaN,NaN,RESTON,VA,201903219,7.036899e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y
1,1003000126,7517003643,I20130530000085,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,EMERGENCY MEDICINE ASSOCIATES PA PC,8.022915e+09,182.0,1701 N GEORGE MASON DR,NaN,NaN,ARLINGTON,VA,222053610,7.035586e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y
2,1003000126,7517003643,I20150824000105,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,EMERGENCY MEDICINE ASSOCIATES PA PC,8.022915e+09,182.0,24440 STONE SPRINGS BLVD,NaN,NaN,DULLES,VA,201662247,5.713674e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y
3,1003000126,7517003643,I20150824000105,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,SOUTHEASTERN INTENSIVIST SERVICES PC,9.335152e+09,133.0,1401 JOHNSTON WILLIS DR,NaN,NaN,NORTH CHESTERFIELD,VA,232354730,8.044835e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y
4,1003000126,7517003643,I20150824000105,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,SOUTHEASTERN INTENSIVIST SERVICES PC,9.335152e+09,133.0,411 W RANDOLPH RD,NaN,NaN,HOPEWELL,VA,238602938,8.045412e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y


To ensure all the IDs and the `full_nm` column are good to go, let's check if there were any missing values in the first four columns. We can also take a look at the total number of unique IDs and names there are in the data set. 

In [2]:
# check for missing values
data[['npi', 'ind_pac_id', 'ind_enrl_id', 'full_nm']].isna().sum()

npi            0
ind_pac_id     0
ind_enrl_id    0
full_nm        0
dtype: int64

In [3]:
# no missing values! No let's take a look at how many unique values we have in each column
for col in data[['npi', 'ind_pac_id', 'ind_enrl_id', 'full_nm']]:
    unique = len(data[col].unique())
    print('There are {} unique values in the {} column.'.format(unique, col))

There are 1103330 unique values in the npi column.
There are 1103330 unique values in the ind_pac_id column.
There are 1147121 unique values in the ind_enrl_id column.
There are 1072060 unique values in the full_nm column.


As we can see there is difference in the number of unique observations amongst the first four columns. The reason why this is important to look into further is that we need to use a column to group observations. In previous notebooks, we have seen that the same physician will have multiple rows, just take a look at the first five rows printed out above! 

However, since there are no missing values in any of these columns we can move onto cleaning the others.

In [4]:
# take a look at how many missing values there are in gender column
print('There are {} missing values in the gender column'.format(data['gndr'].isna().sum()))

There are 0 missing values in the gender column


In [5]:
# get the relative value counts in the gender column
round((data['gndr'].value_counts(normalize=True)) * 100, 3)

M    55.359
F    44.641
Name: gndr, dtype: float64

Our physician data set is approximately 55% male and ~45% female. With no missing values lets move onto the `cred` column.

In [6]:
# take a look at the absolute number of missing values in credential column
data['cred'].isna().sum()

0

In [7]:
# what percentage of the cred column is missing values?
print('Approximately {}% of the cred column is missing.'.format(round((data['cred'].isna().mean()) * 100, 3)))

Approximately 0.0% of the cred column is missing.


In [8]:
# how many unique values are there in the cred column?
print('There are {} unique values in the credential column.'.format(len(data['cred'].unique())))

There are 22 unique values in the credential column.


In [9]:
# what are the unique values in the credential column?
for cred in data['cred'].unique():
    print(cred)

Not Listed
PA
PT
CNS
MD
CSW
CNA
DC
NP
OD
CP
DO
CNM
AU
OT
MNT
DPM
AA
DDS
PSY
DDM
SCW


Above are all the unique values in the credential column. The first one is `nan` which means '_not a number_' and is something we'll address in a little bit. There is two more issues: the first being that we need to do some research about what each of these mean; secondly, there appears to be a leading whitespcate in some of the values. 

Before we start our research, let's go ahead and replace `NaN` but how do we want to approach this? We don't want to replace it with `None` as being a medical professional virtually always requires some type of certification to ensure that particular individual has the requisite skills to care for patients. However, it is not plausible to research each and every physician to determine what school they graduated from. 

However, before we go any further lets check out the `grd_yr` column to see if there are any missing values. Essentially, `grd_yr` can serve as a proxy for `cred` in that we can firt confirm that an individual graduated from a program in the first place. If there are no missing values, this represents the case that all graduated from somewhere, we just don't know where that somewhere is. 

In [10]:
# check grad_yr for missing values
print('There are {} missing values in the graduation year column.'.format(data['grd_yr'].isna().sum()))

There are 0 missing values in the graduation year column.


In [11]:
# print out the unique graduation years
for year in sorted(data['grd_yr'].unique()):
    print(year)

0
1944
1945
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


A-ha! Despite having no missing values it looks like there are values of `0` in the column as indicated by a print out of all the unique values! Now I remember that a few notebooks back I looked into the `grd_yr` column and reassigned the missing values to `0`. This was done in an attempt to preserve any other information that might be useful in that particular observation. 

Let's take a quick look and see how many observations have a value of `0` for `grd_yr`.

In [12]:
# how many observations have a value of 0 in grd_yr?
zero = len(data[data['grd_yr'] == 0])
print('There are {} observations in the grd_yr column that have a value of 0.'.format(zero))

There are 5199 observations in the grd_yr column that have a value of 0.


In [13]:
# as a reminder, how many observations are missing in the cred column?
print('There are {} missing values in the credential column.'.format(data['cred'].isna().sum()))

There are 0 missing values in the credential column.


As we can see, there is a disjoint between the graduation date and credential data. The vast majority of observations seem to have a graduation year, with only _~0.24%_ having a missing value (i.e., corresponding to a value of `0`. In comparison to the `cred` column, missing values comprise _~68.98%_ of observations. 

What is this telling us? That virtually all the physicians graduated from _some institution_ but we aren't sure exactly which institution that is. Now, what can we do? It is highly unfeasible to go through every observation, look up the physician's name, and then determine what credential they graduated with. While it would provide a complete picture of the data, the time one would have to invest is significant, so that throws this option out of the window. 

What can we do? Something similar to what we did in the `grd_yr` column when we labeled the missing values as `0`. In this particular case, though we can substitute something like `not listed` seeing as the data type is string-oriented. That way we can keep the other information for that physician; plus if we take a look at the `pri_spec` column, which gives us the primary specialty of that physician, we can perhaps get a rough inference of what that particular physician may have a credential in. 

In [14]:
# example of how to fill na's 
data['cred'].fillna('Not Listed')[:5]

0    Not Listed
1    Not Listed
2    Not Listed
3    Not Listed
4    Not Listed
Name: cred, dtype: object

In [15]:
# lets go ahead and apply the above to the credential column
data['cred'] = data['cred'].fillna('Not Listed'); data['cred'][:5]

0    Not Listed
1    Not Listed
2    Not Listed
3    Not Listed
4    Not Listed
Name: cred, dtype: object

In [16]:
data.head()

,npi,ind_pac_id,ind_enrl_id,full_nm,gndr,cred,med_sch,grd_yr,pri_spec,sec_spec_1,sec_spec_2,sec_spec_3,sec_spec_4,sec_spec_all,org_lgl_nm,org_pac_id,num_org_mem,adr_ln_1,adr_ln_2,ln_2_sprs,cty,st,zip,phn_numbr,hosp_afl_1,hosp_afl_lbn_1,hosp_afl_2,hosp_afl_lbn_2,hosp_afl_3,hosp_afl_lbn_3,hosp_afl_4,hosp_afl_lbn_4,hosp_afl_5,hosp_afl_lbn_5,assgn
0,1003000126,7517003643,I20130530000085,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,EMERGENCY MEDICINE ASSOCIATES PA PC,8.022915e+09,182.0,1850 TOWN CTR PKWY,NaN,NaN,RESTON,VA,201903219,7.036899e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y
1,1003000126,7517003643,I20130530000085,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,EMERGENCY MEDICINE ASSOCIATES PA PC,8.022915e+09,182.0,1701 N GEORGE MASON DR,NaN,NaN,ARLINGTON,VA,222053610,7.035586e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y
2,1003000126,7517003643,I20150824000105,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,EMERGENCY MEDICINE ASSOCIATES PA PC,8.022915e+09,182.0,24440 STONE SPRINGS BLVD,NaN,NaN,DULLES,VA,201662247,5.713674e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y
3,1003000126,7517003643,I20150824000105,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,SOUTHEASTERN INTENSIVIST SERVICES PC,9.335152e+09,133.0,1401 JOHNSTON WILLIS DR,NaN,NaN,NORTH CHESTERFIELD,VA,232354730,8.044835e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y
4,1003000126,7517003643,I20150824000105,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,SOUTHEASTERN INTENSIVIST SERVICES PC,9.335152e+09,133.0,411 W RANDOLPH RD,NaN,NaN,HOPEWELL,VA,238602938,8.045412e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y


Columns `npi` through `grd_yr` have been addressed; now, let us take a look at the `pri_spec` column.

In [17]:
# how many missing values are there in the pri_spec column?
print('There are {} missing values in the primary specialty column.'.format(data['pri_spec'].isna().sum()))

There are 0 missing values in the primary specialty column.


In [18]:
# how many unique values are there in the pri_spec column?
print('Number of unique values in primary specialty column: {}'.format(len(data['pri_spec'].unique())))

Number of unique values in primary specialty column: 83


With 83 unique values, we have quite a few specialty's listed. Lets print out the unique values and then get the value counts for each.

In [19]:
# print out the unqiue specialties in the pri_spec column, sorted
for specialty in sorted(data['pri_spec'].unique()):
    print(specialty)

ADDICTION MEDICINE
ADVANCED HEART FAILURE AND TRANSPLANT CARDIOLOGY
ALLERGY/IMMUNOLOGY
ANESTHESIOLOGY
ANESTHESIOLOGY ASSISTANT
CARDIAC ELECTROPHYSIOLOGY
CARDIAC SURGERY
CARDIOVASCULAR DISEASE (CARDIOLOGY)
CERTIFIED CLINICAL NURSE SPECIALIST (CNS)
CERTIFIED NURSE MIDWIFE (CNM)
CERTIFIED REGISTERED NURSE ANESTHETIST (CRNA)
CHIROPRACTIC
CLINICAL SOCIAL WORKER
COLORECTAL SURGERY (PROCTOLOGY)
CRITICAL CARE (INTENSIVISTS)
DENTIST
DERMATOLOGY
DIAGNOSTIC RADIOLOGY
EMERGENCY MEDICINE
ENDOCRINOLOGY
FAMILY MEDICINE
GASTROENTEROLOGY
GENERAL PRACTICE
GENERAL SURGERY
GERIATRIC MEDICINE
GERIATRIC PSYCHIATRY
GYNECOLOGICAL ONCOLOGY
HAND SURGERY
HEMATOLOGY
HEMATOLOGY/ONCOLOGY
HEMATOPOIETIC CELL TRANSPLANTATION AND CELLULAR TH
HOSPICE/PALLIATIVE CARE
HOSPITALIST
INFECTIOUS DISEASE
INTERNAL MEDICINE
INTERVENTIONAL CARDIOLOGY
INTERVENTIONAL PAIN MANAGEMENT
INTERVENTIONAL RADIOLOGY
MAXILLOFACIAL SURGERY
MEDICAL GENETICS AND GENOMICS
MEDICAL ONCOLOGY
MEDICAL TOXICOLOGY
NEPHROLOGY
NEUROLOGY
NEUROPSYCHIATRY
NE

Nothing too out of the ordinary with the specialties listed above. However, there is something to perhaps explore a little further. Near the end there are two values - `UNDEFINED NON-PHYSICIAN TYPE (SPECIFY)` & `UNDEFINED PHYSICIAN TYPE (SPECIFY)` that are essentially flagging unknown values. Let's take a quick detour and explore these specific values before going any further. 

In [20]:
# how many values in pri_spec column are of the undefined non-physician type?
len(data[data['pri_spec'] == 'UNDEFINED NON-PHYSICIAN TYPE (SPECIFY)'])

18

In [21]:
# how many values in pri_spec column are of the undefined physician type?
len(data[data['pri_spec'] == 'UNDEFINED PHYSICIAN TYPE (SPECIFY)'])

612

It looks like there are very few of these observations when compared to the overall data set (which is 2.1M rows). However, lets keep this in mind going forward as perhaps we might be able to find more information in another column, perhaps in one of the secondary specialty columns.

Next, we'll get back to the higher level overview of the `pri_spec` column by getting the value counts for each unique value in the column.

In [22]:
# return absolute value counts for top 20 values in primary specialty column
data['pri_spec'].value_counts()[:20]

NURSE PRACTITIONER                               235345
PHYSICIAN ASSISTANT                              203371
INTERNAL MEDICINE                                182186
FAMILY MEDICINE                                  158359
DIAGNOSTIC RADIOLOGY                             156303
PHYSICAL THERAPY                                  84096
CERTIFIED REGISTERED NURSE ANESTHETIST (CRNA)     79246
ANESTHESIOLOGY                                    75169
CARDIOVASCULAR DISEASE (CARDIOLOGY)               66414
OBSTETRICS/GYNECOLOGY                             59125
CLINICAL SOCIAL WORKER                            49860
ORTHOPEDIC SURGERY                                45332
OPTOMETRY                                         44946
GENERAL SURGERY                                   44396
CHIROPRACTIC                                      42595
PSYCHIATRY                                        39718
OPHTHALMOLOGY                                     34855
NEUROLOGY                                       

Quickly, what is wrong with the above?

YOU FORGOT THAT THERE IS DUPLICATE ENTRIES, i.e. there are physicians that are listed multiple times and will thus inflate the numbers for whatever the `pri_spec` is! 

To get a more accurate picture we would first need to groupby the physician and then we can return a more accurate value count.

In [23]:
# group by physician (condensing down duplicate physicians) and grab their associated pri_spec
data.groupby('full_nm')['pri_spec'].first()[:10]

full_nm
A  MIRANDA                       MEDICAL ONCOLOGY
A  NARASIMHA RAO                        NEUROLOGY
A ALAN SEMINE                DIAGNOSTIC RADIOLOGY
A ANANTH  RAMAN                 PULMONARY DISEASE
A ANDREW RUDMANN                INTERNAL MEDICINE
A B  M MASUDUR  RAHMAN            FAMILY MEDICINE
A BENEDICT  COSIMI              SURGICAL ONCOLOGY
A BETHEL  GILBERT          CLINICAL SOCIAL WORKER
A BRANT LIPSCOMB JR.           ORTHOPEDIC SURGERY
A C BADDER                       PHYSICAL THERAPY
Name: pri_spec, dtype: object

In [24]:
# make a dataframe of the above and reset the index
pri_spec_df = pd.DataFrame(data.groupby('full_nm')['pri_spec'].first()).reset_index(); pri_spec_df.head()

,full_nm,pri_spec
0,A MIRANDA,MEDICAL ONCOLOGY
1,A NARASIMHA RAO,NEUROLOGY
2,A ALAN SEMINE,DIAGNOSTIC RADIOLOGY
3,A ANANTH RAMAN,PULMONARY DISEASE
4,A ANDREW RUDMANN,INTERNAL MEDICINE


In [25]:
# adjust column names
pri_spec_df.columns = ['name', 'primary_specialty']

In [26]:
# now return value count in primary specialty column, similar to how we did above
pri_spec_df['primary_specialty'].value_counts()[:10]

NURSE PRACTITIONER                               130090
INTERNAL MEDICINE                                 89304
FAMILY MEDICINE                                   82162
PHYSICIAN ASSISTANT                               73635
PHYSICAL THERAPY                                  57298
CERTIFIED REGISTERED NURSE ANESTHETIST (CRNA)     41544
CLINICAL SOCIAL WORKER                            39863
CHIROPRACTIC                                      39173
ANESTHESIOLOGY                                    37146
OPTOMETRY                                         31212
Name: primary_specialty, dtype: int64

As we can see, by reformatting the data to condense duplicate entries down to one for each physician in the data, we cut many of the values from before nearly in half! This is the more accurate count of the primary specialty for each person in the data set. We'll take note of this for later use however since this column - i.e., `pri_spec` in original `data` DataFrame - is void of missing values we can move forward with our cleaning process to the columns associated with secondary specialties. 

In [27]:
# check the missing values in the columns associated with second specialties
data[['sec_spec_1', 'sec_spec_2', 'sec_spec_3', 'sec_spec_4', 'sec_spec_all']].isna().sum()

sec_spec_1      0
sec_spec_2      0
sec_spec_3      0
sec_spec_4      0
sec_spec_all    0
dtype: int64

In [28]:
# same as above but relayed as a percentage of the column
round((data[['sec_spec_1', 'sec_spec_2', 'sec_spec_3', 'sec_spec_4', 'sec_spec_all']].isna().mean()) * 100, 3)

sec_spec_1      0.0
sec_spec_2      0.0
sec_spec_3      0.0
sec_spec_4      0.0
sec_spec_all    0.0
dtype: float64

It appears that a very large percentage of all these columns don't have any values. Upon a second look though, it appears that we might have a column - `sec_spec_all` - that combines all the secondary specialties into one. Let's check it out further to either confirm or refute this.

In [29]:
# loop through all the secondary specialty columns to see how many unique values they have
for col in data[['sec_spec_1', 'sec_spec_2', 'sec_spec_3', 'sec_spec_4', 'sec_spec_all']]:
    unique = len(data[col].unique())
    print('Number of unique values in {}: {}'.format(col, unique))

Number of unique values in sec_spec_1: 69
Number of unique values in sec_spec_2: 64
Number of unique values in sec_spec_3: 56
Number of unique values in sec_spec_4: 43
Number of unique values in sec_spec_all: 1556


This is further evidence towards the `sec_spec_all` column containing all the combinations of all the physicians secondary specialties. Let's dive further into each one through as it might gives us some clues as to what are the most common secondary specialties that physicians have.

In [30]:
# remember: we have to reformat the dataframe so that it combines all the physicians and their duplicates
# into one observation
specialty_df = pd.DataFrame(data.groupby('full_nm')[['pri_spec', 
                                                     'sec_spec_1', 
                                                     'sec_spec_2', 
                                                     'sec_spec_3', 
                                                     'sec_spec_4', 
                                                     'sec_spec_all']].first()).reset_index()

In [31]:
# check out the first few lines of the dataframe we just created
specialty_df.head()

,full_nm,pri_spec,sec_spec_1,sec_spec_2,sec_spec_3,sec_spec_4,sec_spec_all
0,A MIRANDA,MEDICAL ONCOLOGY,None,None,None,None,None
1,A NARASIMHA RAO,NEUROLOGY,None,None,None,None,None
2,A ALAN SEMINE,DIAGNOSTIC RADIOLOGY,None,None,None,None,None
3,A ANANTH RAMAN,PULMONARY DISEASE,SLEEP MEDICINE,None,None,None,SLEEP MEDICINE
4,A ANDREW RUDMANN,INTERNAL MEDICINE,None,None,None,None,None


In [32]:
# loop through each numbered secondary specialty column and retrieve top 10 value counts
for col in specialty_df[['sec_spec_1', 'sec_spec_2', 'sec_spec_3', 'sec_spec_4']]:
    print('Top 10 Value Counts of {} column'.format(col))
    print(' ')
    print(specialty_df[col].value_counts()[:10])
    print('-' * 30)

Top 10 Value Counts of sec_spec_1 column
 
None                                   944588
INTERNAL MEDICINE                       44526
CRITICAL CARE (INTENSIVISTS)             8178
PEDIATRIC MEDICINE                       6365
CARDIOVASCULAR DISEASE (CARDIOLOGY)      6232
FAMILY MEDICINE                          4020
GENERAL SURGERY                          3998
GERIATRIC MEDICINE                       3250
EMERGENCY MEDICINE                       3171
PAIN MANAGEMENT                          2872
Name: sec_spec_1, dtype: int64
------------------------------
Top 10 Value Counts of sec_spec_2 column
 
None                       1058939
INTERNAL MEDICINE             4974
PULMONARY DISEASE             1130
PAIN MANAGEMENT                676
PEDIATRIC MEDICINE             596
MEDICAL ONCOLOGY               439
VASCULAR SURGERY               423
SLEEP MEDICINE                 412
NUCLEAR MEDICINE               298
HOSPICE/PALLIATIVE CARE        261
Name: sec_spec_2, dtype: int64
-----------

A quick look yields that `INTERNAL MEDICINE` is a value that is in the top 10 of all of the secondary specialty columns. Let's see what's in store for us in the `sec_spec_all` column.

In [33]:
# get top 20 value counts (normalized to show percentage) of the sec_spec_all column
(specialty_df['sec_spec_all'].value_counts(normalize=True) * 100)[:20]

None                                               88.109621
INTERNAL MEDICINE                                   3.970207
PEDIATRIC MEDICINE                                  0.588307
CRITICAL CARE (INTENSIVISTS)                        0.473201
CARDIOVASCULAR DISEASE (CARDIOLOGY)                 0.441113
GENERAL SURGERY                                     0.344384
FAMILY MEDICINE                                     0.334123
GERIATRIC MEDICINE                                  0.258941
PAIN MANAGEMENT                                     0.257821
EMERGENCY MEDICINE                                  0.254557
SPORTS MEDICINE                                     0.235155
HOSPITALIST                                         0.196911
ANESTHESIOLOGY                                      0.176389
CRITICAL CARE (INTENSIVISTS), INTERNAL MEDICINE     0.169953
PULMONARY DISEASE                                   0.165942
INTERVENTIONAL RADIOLOGY                            0.155122
ORTHOPEDIC SURGERY      

For the most part it appears of the physicians that do have a secondary specialty a large portion tend to have one, `INTERNAL MEDICINE`. Another thing to note is that there is only one entry - `CRITICAL CARE (INTENSIVISTS), INTERNAL MEDICINE` - that lists more than one secondary specialty. This is an indicator that it appears most physicians don't tend to have more than one other specialty in addition to their primary one. 

But lets get back to what we were originally seeking to do: fill in the `NaN`s in the secondary specialty columns. For these columns we can go ahead and replace `NaN` with `None` as it most likely to be the case that a particular physician doesn't have a secondary specialty if there is a value of `NaN`. 

In [34]:
# replace NaN in all the secondary specialty columns with None
data[['sec_spec_1', 'sec_spec_2', 'sec_spec_3', 'sec_spec_4', 'sec_spec_all']] = data[['sec_spec_1', 'sec_spec_2', 'sec_spec_3', 'sec_spec_4', 'sec_spec_all']].fillna('None')

Now that we've cleaned up more than a few columns, lets take a look at the first few rows of `data` and see how it is shaping up.

In [35]:
data.head()

,npi,ind_pac_id,ind_enrl_id,full_nm,gndr,cred,med_sch,grd_yr,pri_spec,sec_spec_1,sec_spec_2,sec_spec_3,sec_spec_4,sec_spec_all,org_lgl_nm,org_pac_id,num_org_mem,adr_ln_1,adr_ln_2,ln_2_sprs,cty,st,zip,phn_numbr,hosp_afl_1,hosp_afl_lbn_1,hosp_afl_2,hosp_afl_lbn_2,hosp_afl_3,hosp_afl_lbn_3,hosp_afl_4,hosp_afl_lbn_4,hosp_afl_5,hosp_afl_lbn_5,assgn
0,1003000126,7517003643,I20130530000085,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,EMERGENCY MEDICINE ASSOCIATES PA PC,8.022915e+09,182.0,1850 TOWN CTR PKWY,NaN,NaN,RESTON,VA,201903219,7.036899e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y
1,1003000126,7517003643,I20130530000085,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,EMERGENCY MEDICINE ASSOCIATES PA PC,8.022915e+09,182.0,1701 N GEORGE MASON DR,NaN,NaN,ARLINGTON,VA,222053610,7.035586e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y
2,1003000126,7517003643,I20150824000105,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,EMERGENCY MEDICINE ASSOCIATES PA PC,8.022915e+09,182.0,24440 STONE SPRINGS BLVD,NaN,NaN,DULLES,VA,201662247,5.713674e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y
3,1003000126,7517003643,I20150824000105,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,SOUTHEASTERN INTENSIVIST SERVICES PC,9.335152e+09,133.0,1401 JOHNSTON WILLIS DR,NaN,NaN,NORTH CHESTERFIELD,VA,232354730,8.044835e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y
4,1003000126,7517003643,I20150824000105,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,SOUTHEASTERN INTENSIVIST SERVICES PC,9.335152e+09,133.0,411 W RANDOLPH RD,NaN,NaN,HOPEWELL,VA,238602938,8.045412e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y


In [36]:
# check the percentage of missing values in the columns we've cleaned so far
data.iloc[:, :14].isna().mean() * 100

npi             0.0
ind_pac_id      0.0
ind_enrl_id     0.0
full_nm         0.0
gndr            0.0
cred            0.0
med_sch         0.0
grd_yr          0.0
pri_spec        0.0
sec_spec_1      0.0
sec_spec_2      0.0
sec_spec_3      0.0
sec_spec_4      0.0
sec_spec_all    0.0
dtype: float64

In [ ]:
# save updates to CSV so that tomorrow we can pick up where we left off
data.to_csv('physician_compare_national-updates-1.csv', index=False)

# _Work Completed (9/30/19)_

Due to time constraints, I'm going to cut off this notebook here. Above we saved our updates into a CSV so that tomorrow we can just read it in via `pandas` and start right back up where we left off. Below is an overview of the steps that still need to be completed:
1. Examine all columns after, and including, `org_lgl_nm`
2. Determine if any cleaning is needed on each respective column
3. If cleaning is necessary, determine best how to address missing/unusual values

# _Work Restarted (10/1/29)_

Another day, more data cleaning! We'll jump right in where we left off and being by exploring the `org_lgl_nm` which is the "legal name of the Group Practice that the individual professional works with - will be blank if the address is not linked to a Group Practice." Key thing to remember that if it is blank there is not a link to a group practice, which will help us fill in the `NaN`s, should there be any.

In [38]:
from datetime import datetime
# current date and time
now = datetime.now()
print("Work restarted: ", now)

Work restarted:  2019-10-01 10:45:00.434774


In [41]:
# initial check for missing values in Group Practice column
print('There are {} missing values in org_lgl_nm column.'.format(data['org_lgl_nm'].isna().sum()))

There are 177997 missing values in org_lgl_nm column.


In [43]:
# how many unique values are there in the Group Practice column?
print('There are {} unique values in the org_lgl_nm column.'.format(len(data['org_lgl_nm'].unique())))

There are 74366 unique values in the org_lgl_nm column.


In [45]:
# let's see if NaN values show up in the top 10 for value counts
data['org_lgl_nm'].value_counts(dropna=False)[:10]

NaN                                             177997
REGENTS OF THE UNIVERSITY OF MICHIGAN            30852
SOUTHERN CALIFORNIA PERMANENTE MEDICAL GROUP     18240
THE CLEVELAND CLINIC FOUNDATION                  13097
PERMANENTE MEDICAL GROUP INC                     12647
PRIVATE DIAGNOSTIC CLINIC PLLC                   12462
UNIVERSITY OF PITTSBURGH PHYSICIANS              12242
NORTH SHORE-LIJ MEDICAL PC                       11005
NEW YORK UNIVERSITY                               9143
FLORIDA CLINICAL PRACTICE ASSOCIATION INC         8397
Name: org_lgl_nm, dtype: int64

`NaN` is the top value! Remember that this represents that it is not linked to a Group Practice so we can go ahead and replace `NaN` with `None`. Below is an example of what we'll be doing for the whole `org_lgl_nm` column; see index 197 and 201 where the corresponding values have been converted from `NaN` to `None`.

In [50]:
# example of what we'll be doing
data['org_lgl_nm'].fillna('None')[190:210]

190            ADVOCATE HEALTH AND HOSPITALS CORPORATION
191    PROFESSIONAL SERVICES IN CARDIOLOGY PHYSICIANS...
192                                  ADNAN AFZAL, MD, PA
193                    YAKIMA VALLEY FARM WORKERS CLINIC
194                    YAKIMA VALLEY FARM WORKERS CLINIC
195                    YAKIMA VALLEY FARM WORKERS CLINIC
196                    YAKIMA VALLEY FARM WORKERS CLINIC
197                                                 None
198                                 ALLINA HEALTH SYSTEM
199                       HENNEPIN HEALTHCARE SYSTEM INC
200                        BERGEN ANESTHESIA GROUP, P.C.
201                                                 None
202                                            UPMC KANE
203                                            UPMC KANE
204                               OHIOHEALTH CORPORATION
205                               OHIOHEALTH CORPORATION
206                               OHIOHEALTH CORPORATION
207                            

In [51]:
# apply the above to data set
data['org_lgl_nm'] = data['org_lgl_nm'].fillna('None')

Now we're going to move onto the `org_pac_id` column; below is the corresponding description from [data.medicare.gov](https://data.medicare.gov/Physician-Compare/Physician-Compare-National-Downloadable-File/mj5m-pzi6):
- "_Unique Group Practice ID assigned by PECOS to the Group Practice that the individual professional works with- will be blank if the address is not linked to a Group Practice_"

In theory, because this column is linked with the one we just worked with - `org_lgl_nm` - we should see similarities in the number of `NaN`s and unique values.

In [53]:
# how many missing values are in the org_pac_id column
print('There are {} missing values in org_pac_id column.'.format(data['org_pac_id'].isna().sum()))

There are 177997 missing values in org_pac_id column.


In [56]:
# how many unique values are there in the org_pac_id column?
print('There are {} unique values in the org_pac_id column.'.format(len(data['org_pac_id'].unique())))

There are 74998 unique values in the org_pac_id column.


Quick observation: there are slightly more (~600) unique values in `org_pac_id` than in the `org_lgl_nm` column. It could be similar to what was going on with the ID columns pertaining to the physicians in that certain Group Practices may have multiple IDs. 

In [63]:
# let's see if NaN values show up in the top 10 for value counts
data['org_pac_id'].value_counts(dropna=False).head(10)

NaN             177997
3.779497e+09     30852
6.002729e+09     18240
1.850204e+09     13097
8.921910e+09     12647
1.355254e+09     12462
8.729990e+09     12242
3.375702e+09     11005
1.355232e+09      9143
3.451463e+08      8397
Name: org_pac_id, dtype: int64

`NaN` is again the leader. However, we cannot use `None` here because they data in this column is a `int64` numeric type. What we can do then, as a proxy for `None` is to use `0`. Let's go ahead and see what happens when we apply this change.

In [66]:
# fill NaN with 0 and return top 10 value coounts
data['org_pac_id'].fillna(0).value_counts().head(10)

0.000000e+00    177997
3.779497e+09     30852
6.002729e+09     18240
1.850204e+09     13097
8.921910e+09     12647
1.355254e+09     12462
8.729990e+09     12242
3.375702e+09     11005
1.355232e+09      9143
3.451463e+08      8397
Name: org_pac_id, dtype: int64

Good, all of the `NaN`s have been replaced by `0`. We'll need to take note though that there was that discrepancy between the number of unique values in the `org_lgl_nm` column and the `org_pac_id` column and explore it further. For now though lets move onto the `num_org_mem` column, which is linked to the previous two columns as well. Below is the columns description:

- "_Total number of individual professionals affiliated with the Group Practice based on Group Practice PAC ID_"

From the description, this number has to do with how many professionals are in each Group Practice. So if an individual physician is not part of a Group Practice, it could be said that there is only 1 in that group (i.e. just that individual physician). However, let's be sure to take a look at the value counts because this may be one of the values already, and if it is, then we'll have to find another way to address the `NaN`s.

In [70]:
# how many missing values are in the num_org_mem column?
print('There are {} missing values in the num_org_mem column.'.format(data['num_org_mem'].isna().sum()))

There are 177997 missing values in the num_org_mem column.


In [72]:
# how many unique values are there in the num_org_mem column?
print('There are {} unique values in the num_org_mem column.'.format(len(data['num_org_mem'].unique())))

There are 649 unique values in the num_org_mem column.


In [73]:
# let's see if NaN values show up in the top 10 for value counts
data['num_org_mem'].value_counts(dropna=False).head(10)

NaN       177997
2.0        56170
3.0        42884
4.0        36351
5.0        31302
2686.0     30852
6.0        26382
7.0        23366
8.0        22634
9.0        19526
Name: num_org_mem, dtype: int64

The value count above gives us a few insights into the `num_org_mem` column. The first one is that we need to change this column from a `float` to an `int` type; the second one is that since there are no values of `1` in this column (representing a solo physician), we can use that to represent physician's that are not associated with any particular Group Practice; lastly, we can see that outside the value of `2686.0` it looks like most physicians tend to be part of Group Practices that are relatively small (_< 10_). This will be something to explore further later on. For now though lets replace the `NaN` with `1` and remove the decimal.

In [77]:
# show what missing values looked like before
data['num_org_mem'][2210761:2210776]

2210761    1079.0
2210762       NaN
2210763       3.0
2210764       3.0
2210765       4.0
2210766    8118.0
2210767    8118.0
2210768    8118.0
2210769    8118.0
2210770    1147.0
2210771       9.0
2210772       9.0
2210773       NaN
2210774       NaN
2210775     105.0
Name: num_org_mem, dtype: float64

In [78]:
# show the after (i.e. what we are doing with missing values in column)
data['num_org_mem'].fillna(1)[2210761:2210776]

2210761    1079.0
2210762       1.0
2210763       3.0
2210764       3.0
2210765       4.0
2210766    8118.0
2210767    8118.0
2210768    8118.0
2210769    8118.0
2210770    1147.0
2210771       9.0
2210772       9.0
2210773       1.0
2210774       1.0
2210775     105.0
Name: num_org_mem, dtype: float64

In [79]:
# apply the above method to the num_org_mem column
data['num_org_mem'] = data['num_org_mem'].fillna(1)

In [82]:
# example of converting to int type (i.e. no decimals since you can't have a portion of a person!)
data['num_org_mem'].astype(int)[:5]

0    182
1    182
2    182
3    133
4    133
Name: num_org_mem, dtype: int64

In [83]:
# apply the above method to the column
data['num_org_mem'] = data['num_org_mem'].astype(int)

In [84]:
# final check of the column
data['num_org_mem'].value_counts(dropna=False).head(10)

1       177997
2        56170
3        42884
4        36351
5        31302
2686     30852
6        26382
7        23366
8        22634
9        19526
Name: num_org_mem, dtype: int64

Next we're going to move into the columns pertaining to addresses. Lets take a look at the number of missing values in each of them.

In [86]:
# gather counts for missing values for columns linked to address information
for col in data[['adr_ln_1', 'adr_ln_2', 'ln_2_sprs', 'cty', 'st', 'zip']]:
    missing = data[col].isna().sum()
    print('There are {} missing values in the {} column'.format(missing, col))

There are 0 missing values in the adr_ln_1 column
There are 1440523 missing values in the adr_ln_2 column
There are 2084103 missing values in the ln_2_sprs column
There are 0 missing values in the cty column
There are 0 missing values in the st column
There are 0 missing values in the zip column


It looks like we're good to go in regards to the primary address columns: `adr_ln_1`, `cty`, `st` and `zip`. The first column with missing values - `adr_ln_2` - is mostly supplementary; however, `ln_2_sprs` is its own entity and represents if that particular observations address may be incomplete. What should we do here? Well we can first fill in the `NaN`s in the `adr_ln_2` columns with a blank `""`, and then we can look further into the `ln_2_sprs` to see how many incomplete addresses we are dealing with. 

After that, we can then look into combining address information into one column, as `pandas` makes it pretty easy to combine columns.

In [90]:
# replace NaN with blank input for adr_ln_2 column
data['adr_ln_2'] = data['adr_ln_2'].fillna("")

In [91]:
# take a look at the value count in the ln_2_sprs column
data['ln_2_sprs'].value_counts(dropna=False)

NaN    2084103
Y       126687
Name: ln_2_sprs, dtype: int64

In [96]:
# same as above but presented as a percentage of the values
round((data['ln_2_sprs'].value_counts(dropna=False, normalize=True) * 100), 2)

NaN    94.27
Y       5.73
Name: ln_2_sprs, dtype: float64

It looks like most of the addresses are good, with only approximately ~5% potentially being incomplete. Let's take a quick look at these 'incomplete' addresses to see what they may look like.

In [98]:
# replace NaN with N (represents No)
data['ln_2_sprs'] = data['ln_2_sprs'].fillna('N'); data['ln_2_sprs'][:10]

0    N
1    N
2    N
3    N
4    N
5    N
6    Y
7    N
8    Y
9    N
Name: ln_2_sprs, dtype: object

In [99]:
# same as above but presented as a percentage of the values
round((data['ln_2_sprs'].value_counts(dropna=False, normalize=True) * 100), 2)

N    94.27
Y     5.73
Name: ln_2_sprs, dtype: float64

In [102]:
# subset of observations that have Y for potentially incomplete addresses
data[data['ln_2_sprs'] == 'Y'].head()

,npi,ind_pac_id,ind_enrl_id,full_nm,gndr,cred,med_sch,grd_yr,pri_spec,sec_spec_1,sec_spec_2,sec_spec_3,sec_spec_4,sec_spec_all,org_lgl_nm,org_pac_id,num_org_mem,adr_ln_1,adr_ln_2,ln_2_sprs,cty,st,zip,phn_numbr,hosp_afl_1,hosp_afl_lbn_1,hosp_afl_2,hosp_afl_lbn_2,hosp_afl_3,hosp_afl_lbn_3,hosp_afl_4,hosp_afl_lbn_4,hosp_afl_5,hosp_afl_lbn_5,assgn
6,1003000134,4284706367,I20080707000385,THOMAS L CIBULL,M,Not Listed,UNIVERSITY OF KENTUCKY COLLEGE OF MEDICINE,2003,PATHOLOGY,None,None,None,None,None,NORTHSHORE UNIVERSITY HEALTHSYSTEM FACULTY PRA...,2.163335e+09,1286,2180 PFINGSTEN RD 1ST FLOOR,,Y,GLENVIEW,IL,600261301,NaN,140010.0,NORTHSHORE UNIVERSITY HEALTHSYSTEM - EVANSTON ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
8,1003000134,4284706367,I20080707000385,THOMAS L CIBULL,M,Not Listed,UNIVERSITY OF KENTUCKY COLLEGE OF MEDICINE,2003,PATHOLOGY,None,None,None,None,None,NORTHSHORE UNIVERSITY HEALTHSYSTEM FACULTY PRA...,2.163335e+09,1286,777 PARK AVE W,,Y,HIGHALND PARK,IL,600352433,NaN,140010.0,NORTHSHORE UNIVERSITY HEALTHSYSTEM - EVANSTON ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
11,1003000134,4284706367,I20080707000385,THOMAS L CIBULL,M,Not Listed,UNIVERSITY OF KENTUCKY COLLEGE OF MEDICINE,2003,PATHOLOGY,None,None,None,None,None,NORTHSHORE UNIVERSITY HEALTHSYSTEM FACULTY PRA...,2.163335e+09,1286,9650 GROSS POINT RD,,Y,SKOKIE,IL,600761214,NaN,140010.0,NORTHSHORE UNIVERSITY HEALTHSYSTEM - EVANSTON ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
14,1003000142,9931380672,I20110801000609,RASHID KHALIL,M,Not Listed,OTHER,1999,ANESTHESIOLOGY,None,None,None,None,None,"MERCY HEALTH PHYSICIANS - NORTH, LLC.",2.668522e+09,376,2213 CHERRY ST,,Y,TOLEDO,OH,436082603,NaN,360262.0,MERCY ST ANNE HOSPITAL,360112.0,MERCY ST VINCENT MEDICAL CENTER,360081.0,MERCY ST CHARLES HOSPITAL,NaN,NaN,NaN,NaN,Y
15,1003000142,9931380672,I20110801000609,RASHID KHALIL,M,Not Listed,OTHER,1999,ANESTHESIOLOGY,None,None,None,None,None,"MERCY HEALTH PHYSICIANS - NORTH, LLC.",2.668522e+09,376,2222 CHERRY ST,,Y,TOLEDO,OH,436082675,NaN,360262.0,MERCY ST ANNE HOSPITAL,360112.0,MERCY ST VINCENT MEDICAL CENTER,360081.0,MERCY ST CHARLES HOSPITAL,NaN,NaN,NaN,NaN,Y


From a quick glance, it appears that what might be going on is the case where the physician's exact location is not known. For example, the primary address could be for a hospital's location, but does not include the physician's specific office number (or a similar situation). Let's go ahead and combine the two address columns - `adr_ln_1` and `adr_ln_2` into one column. We'll keep the `ln_2_sprs` column just in case we need it down the road though.

In [110]:
# combine address columns together into a full address column
full_adr = data['adr_ln_1'].astype(str) + ' ' + data['adr_ln_2'].astype(str)

In [111]:
# insert the combined addresses into data at specified column index
data.insert(19, column='full_adr', value=full_adr)

In [113]:
data.head()

,npi,ind_pac_id,ind_enrl_id,full_nm,gndr,cred,med_sch,grd_yr,pri_spec,sec_spec_1,sec_spec_2,sec_spec_3,sec_spec_4,sec_spec_all,org_lgl_nm,org_pac_id,num_org_mem,adr_ln_1,adr_ln_2,full_adr,ln_2_sprs,cty,st,zip,phn_numbr,hosp_afl_1,hosp_afl_lbn_1,hosp_afl_2,hosp_afl_lbn_2,hosp_afl_3,hosp_afl_lbn_3,hosp_afl_4,hosp_afl_lbn_4,hosp_afl_5,hosp_afl_lbn_5,assgn
0,1003000126,7517003643,I20130530000085,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,EMERGENCY MEDICINE ASSOCIATES PA PC,8.022915e+09,182,1850 TOWN CTR PKWY,,1850 TOWN CTR PKWY,N,RESTON,VA,201903219,7.036899e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y
1,1003000126,7517003643,I20130530000085,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,EMERGENCY MEDICINE ASSOCIATES PA PC,8.022915e+09,182,1701 N GEORGE MASON DR,,1701 N GEORGE MASON DR,N,ARLINGTON,VA,222053610,7.035586e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y
2,1003000126,7517003643,I20150824000105,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,EMERGENCY MEDICINE ASSOCIATES PA PC,8.022915e+09,182,24440 STONE SPRINGS BLVD,,24440 STONE SPRINGS BLVD,N,DULLES,VA,201662247,5.713674e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y
3,1003000126,7517003643,I20150824000105,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,SOUTHEASTERN INTENSIVIST SERVICES PC,9.335152e+09,133,1401 JOHNSTON WILLIS DR,,1401 JOHNSTON WILLIS DR,N,NORTH CHESTERFIELD,VA,232354730,8.044835e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y
4,1003000126,7517003643,I20150824000105,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,SOUTHEASTERN INTENSIVIST SERVICES PC,9.335152e+09,133,411 W RANDOLPH RD,,411 W RANDOLPH RD,N,HOPEWELL,VA,238602938,8.045412e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y


In [114]:
# drop the two address columns
data.drop(labels=['adr_ln_1', 'adr_ln_2'], axis=1, inplace=True)

In [115]:
# double check
data.head()

,npi,ind_pac_id,ind_enrl_id,full_nm,gndr,cred,med_sch,grd_yr,pri_spec,sec_spec_1,sec_spec_2,sec_spec_3,sec_spec_4,sec_spec_all,org_lgl_nm,org_pac_id,num_org_mem,full_adr,ln_2_sprs,cty,st,zip,phn_numbr,hosp_afl_1,hosp_afl_lbn_1,hosp_afl_2,hosp_afl_lbn_2,hosp_afl_3,hosp_afl_lbn_3,hosp_afl_4,hosp_afl_lbn_4,hosp_afl_5,hosp_afl_lbn_5,assgn
0,1003000126,7517003643,I20130530000085,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,EMERGENCY MEDICINE ASSOCIATES PA PC,8.022915e+09,182,1850 TOWN CTR PKWY,N,RESTON,VA,201903219,7.036899e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y
1,1003000126,7517003643,I20130530000085,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,EMERGENCY MEDICINE ASSOCIATES PA PC,8.022915e+09,182,1701 N GEORGE MASON DR,N,ARLINGTON,VA,222053610,7.035586e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y
2,1003000126,7517003643,I20150824000105,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,EMERGENCY MEDICINE ASSOCIATES PA PC,8.022915e+09,182,24440 STONE SPRINGS BLVD,N,DULLES,VA,201662247,5.713674e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y
3,1003000126,7517003643,I20150824000105,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,SOUTHEASTERN INTENSIVIST SERVICES PC,9.335152e+09,133,1401 JOHNSTON WILLIS DR,N,NORTH CHESTERFIELD,VA,232354730,8.044835e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y
4,1003000126,7517003643,I20150824000105,ARDALAN ENKESHAFI,M,Not Listed,OTHER,1994,INTERNAL MEDICINE,None,None,None,None,None,SOUTHEASTERN INTENSIVIST SERVICES PC,9.335152e+09,133,411 W RANDOLPH RD,N,HOPEWELL,VA,238602938,8.045412e+09,490112.0,CJW MEDICAL CENTER,210028.0,MEDSTAR SAINT MARY'S HOSPITAL,NaN,NaN,NaN,NaN,NaN,NaN,Y


The last two columns we'll review today are `cty` and `st`. The values represent the Group Practice/individual's city and state, respectively.

In [116]:
# check missing values in city and state columns
data[['cty', 'st']].isna().sum()

cty    0
st     0
dtype: int64

In [120]:
# what are the top 10 values for city?
(data['cty'].value_counts(dropna=False, normalize=True) * 100).head(10)

ANN ARBOR       1.344361
HOUSTON         1.303109
NEW YORK        1.252358
CHICAGO         0.840062
LOS ANGELES     0.814279
PHILADELPHIA    0.766513
PITTSBURGH      0.719019
BALTIMORE       0.672158
ROCHESTER       0.628644
DALLAS          0.614893
Name: cty, dtype: float64

In [123]:
# what are the top 10 states?
(data['st'].value_counts(dropna=False, normalize=True) * 100).head(10)

CA    8.242031
NY    6.655675
TX    6.420646
FL    5.537116
PA    5.341710
MI    4.743463
IL    4.218266
OH    4.023494
NC    3.683615
MA    3.067094
Name: st, dtype: float64

In [127]:
# what data types are in these two columns?
for col in data[['cty', 'st']]:
    dtype = data[col].dtype
    print('The data type for "{}" is: {}'.format(col, dtype))

The data type for "cty" is: object
The data type for "st" is: object


In [128]:
# save updates to CSV
data.to_csv('physician_compare_national-updates-2.csv', index=False)

In [129]:
from datetime import datetime
# current date and time
now = datetime.now()
print("Work ended: ", now)

Work ended:  2019-10-01 13:34:47.500785


# _Review of Today's (10/1/29) Work_

Today we got the chance to edit up to the `st` column; tomorrow we'll continue and hopefully clean up the remaining columns in `data`. Again, we'll save `data` to a CSV so we can pick up right where we left off.